#Building an image retrieval system with deep features


#Fire up GraphLab Create

In [1]:
import graphlab

#Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [2]:
image_train = graphlab.SFrame('image_train_data/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1526509907.log


This non-commercial license of GraphLab Create for academic use is assigned to gblazer@adobe.com and will expire on February 22, 2019.


#Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [ ]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [3]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


#Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [4]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


#Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [18]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [7]:
knn_model.query(cat)

PROGRESS: Starting pairwise querying...
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 7.97ms       |
PROGRESS: | Done         |         | 100         | 62.996ms     |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


We are going to create a simple function to view the nearest neighbors to save typing:

In [32]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [33]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

NameError: name 'knn_model' is not defined

In [34]:
cat_neighbors['image'].show()

NameError: name 'cat_neighbors' is not defined

Very cool results showing similar cats.

##Finding similar images to a car

In [12]:
car = image_train[8:9]
car['image'].show()

In [31]:
get_images_from_ids(knn_model.query(car))['image'].show()

NameError: name 'knn_model' is not defined

#Just for fun, let's create a lambda to find and show nearest neighbor images

In [23]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [24]:
show_neighbors(8)

NameError: global name 'knn_model' is not defined

In [16]:
show_neighbors(26)

PROGRESS: Starting pairwise querying...
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 14.465ms     |
PROGRESS: | Done         |         | 100         | 69.576ms     |
PROGRESS: +--------------+---------+-------------+--------------+


In [4]:
summary = image_train['label'].sketch_summary()

In [5]:
image_train

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


NameError: name 'label' is not defined

In [9]:
bird = image_train[image_train['label'] == 'bird'] 

In [10]:
dog = image_train[image_train['label'] == 'dog'] 

In [11]:
automobile = image_train[image_train['label'] == 'automobile'] 

In [12]:
bird = image_train[image_train['label'] == 'bird'] 

In [13]:
get = lambda i: image_train[image_train['label'] == i] 

In [14]:
get('bird')

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."
335,Height: 32 Width: 32,bird,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 8.50706672668, 0.0, ...","[160.0, 159.0, 154.0,162.0, 161.0, 156.0, ..."
560,Height: 32 Width: 32,bird,"[1.69159495831, 0.0, 0.0,0.0, 0.0, 0.0, ...","[147.0, 138.0, 88.0,151.0, 142.0, 92.0, ..."
649,Height: 32 Width: 32,bird,"[0.511156201363,0.324165046215, 0.0, ...","[65.0, 127.0, 9.0, 127.0,160.0, 15.0, 159.0, ..."
775,Height: 32 Width: 32,bird,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 10.0127315521, 0.0, ...","[29.0, 41.0, 25.0, 29.0,42.0, 25.0, 28.0, 41.0, ..."
802,Height: 32 Width: 32,bird,"[0.277166724205, 0.0,0.0, 0.0, 0.0, 0.0, ...","[233.0, 230.0, 173.0,222.0, 218.0, 168.0, ..."
975,Height: 32 Width: 32,bird,"[0.0, 0.0336718559265,0.0, 0.645326733589, ...","[59.0, 180.0, 110.0,88.0, 186.0, 117.0, ..."


In [15]:
get('automobile')

id,image,label,deep_features,image_array
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
302,Height: 32 Width: 32,automobile,"[0.583938002586, 0.0,0.0, 0.0, 0.0, 0.0, ...","[64.0, 52.0, 37.0, 85.0,60.0, 40.0, 92.0, 66.0, ..."
312,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0,0.392823398113, 0.0, ...","[124.0, 126.0, 113.0,124.0, 126.0, 113.0, ..."
323,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0,4.42310428619, ...","[241.0, 241.0, 241.0,238.0, 238.0, 238.0, ..."
536,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 8.42903900146, 0.0, ...","[164.0, 154.0, 154.0,128.0, 119.0, 120.0, ..."
593,Height: 32 Width: 32,automobile,"[1.65033948421, 0.0, 0.0,0.0, 0.0, 0.0, ...","[231.0, 222.0, 227.0,232.0, 217.0, 221.0, ..."
962,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0,0.39552795887, 0.0, 0.0, ...","[255.0, 255.0, 255.0,255.0, 255.0, 255.0, ..."
997,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 8.04085636139, 0.0, ...","[145.0, 148.0, 157.0,131.0, 134.0, 145.0, ..."
1421,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.359612941742, ...","[114.0, 95.0, 33.0,118.0, 98.0, 26.0, 91.0, ..."


In [119]:
dog_model = graphlab.nearest_neighbors.create(get('dog'),features=['deep_features'] ,
                                             label='id')

Starting brute force nearest neighbors model training.

In [19]:
cat['image'].show()

In [20]:
dog_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 14.829ms     |

| Done         |         | 100         | 41.414ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,3187,41.0940191828,1
0,13387,42.0746924478,2
0,3616,42.3313388102,3
0,35879,43.0859398027,4
0,47018,43.4137148295,5


In [22]:
get_images_from_ids(dog_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 6.933ms      |

| Done         |         | 100         | 37.373ms     |

+--------------+---------+-------------+--------------+

id,image,label,deep_features,image_array
3187,Height: 32 Width: 32,dog,"[0.0, 0.0305204391479,0.0, 0.0, 0.0, ...","[34.0, 33.0, 30.0, 34.0,33.0, 31.0, 34.0, 33.0, ..."
3616,Height: 32 Width: 32,dog,"[0.0, 1.8823363781,0.442216396332, 0.0, ...","[10.0, 3.0, 3.0, 7.0,2.0, 2.0, 14.0, 11.0, ..."
13387,Height: 32 Width: 32,dog,"[0.366494178772, 0.0,0.0, 0.0, 0.0, 0.0, ...","[255.0, 255.0, 255.0,255.0, 255.0, 255.0, ..."
35879,Height: 32 Width: 32,dog,"[0.0, 1.13229203224, 0.0,0.0, 0.0, 0.0, ...","[227.0, 227.0, 227.0,232.0, 232.0, 232.0, ..."
47018,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 10.9659748077, 0.0, ...","[44.0, 40.0, 41.0, 36.0,30.0, 31.0, 38.0, 30.0, ..."


In [26]:
image_test = graphlab.SFrame('image_test_data/')

In [27]:
image_test[0:1]

id,image,label,deep_features,image_array
0,Height: 32 Width: 32,cat,"[1.13469004631, 0.0, 0.0,0.0, 0.0366497635841, ...","[158.0, 112.0, 49.0,159.0, 111.0, 47.0, ..."


In [29]:
cat_model = graphlab.nearest_neighbors.create(get('cat'),features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [37]:
get_images_from_ids(cat_model.query(image_test[0:1]))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 4.199ms      |

| Done         |         | 100         | 57.66ms      |

+--------------+---------+-------------+--------------+

id,image,label,deep_features,image_array
331,Height: 32 Width: 32,cat,"[0.0, 0.0,0.510963916779, 0.0, ...","[45.0, 65.0, 92.0, 72.0,95.0, 110.0, 106.0, ..."
16289,Height: 32 Width: 32,cat,"[0.964287519455, 0.0,0.0, 0.0, 1.12515509129, ...","[215.0, 219.0, 231.0,215.0, 219.0, 232.0, ..."
25713,Height: 32 Width: 32,cat,"[0.536971271038, 0.0,0.0, 0.0894458889961, ...","[228.0, 222.0, 236.0,224.0, 213.0, 222.0, ..."
32139,Height: 32 Width: 32,cat,"[1.29409468174, 0.0, 0.0,0.513800263405, ...","[217.0, 220.0, 205.0,221.0, 227.0, 218.0, ..."
45646,Height: 32 Width: 32,cat,"[0.983677506447, 0.0,0.0, 0.0, 0.0, ...","[51.0, 42.0, 26.0, 56.0,47.0, 31.0, 59.0, 50.0, ..."


In [41]:
cat_model.query(image_test[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 6.429ms      |

| Done         |         | 100         | 61.027ms     |

+--------------+---------+-------------+--------------+

36.15573070978294

In [42]:
dog_model.query(image_test[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 21.591ms     |

| Done         |         | 100         | 80.52ms      |

+--------------+---------+-------------+--------------+

37.77071136184157

In [43]:
bird_test = image_test[image_test['label'] == 'bird'] 

In [45]:
dog_test = image_test[image_test['label'] == 'dog'] 

In [46]:
cat_test = image_test[image_test['label'] == 'cat'] 

In [47]:
automobile_test = image_test[image_test['label'] == 'automobile'] 

In [50]:
dog_distances = dog_model.query(dog_test, k=1)['distance']

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 64000   | 12.5737     | 255.489ms    |

| Done         | 509000  | 100         | 316.878ms    |

+--------------+---------+-------------+--------------+

In [54]:
bird_model = graphlab.nearest_neighbors.create(get('bird'),features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [55]:
automobile_model = graphlab.nearest_neighbors.create(get('automobile'),features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [56]:
bird_distances = bird_model.query(bird_test, k=1)['distance']

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 60000   | 12.5523     | 229.177ms    |

| Done         | 478000  | 100         | 309.627ms    |

+--------------+---------+-------------+--------------+

In [57]:
automobile_distances = automobile_model.query(automobile_test, k=1)['distance']

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 64000   | 12.5737     | 239.58ms     |

| Done         | 509000  | 100         | 268.233ms    |

+--------------+---------+-------------+--------------+

In [60]:
dog_cat_neighbors = cat_model.query(dog_test, k=1)['distance']

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 64000   | 12.5737     | 245.19ms     |

| Done         | 509000  | 100         | 275.949ms    |

+--------------+---------+-------------+--------------+

In [61]:
dog_bird_neighbors = bird_model.query(dog_test, k=1)['distance']

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 59000   | 12.3431     | 227.027ms    |

| Done         | 478000  | 100         | 276.8ms      |

+--------------+---------+-------------+--------------+

In [62]:
dog_automobile_neighbors = automobile_model.query(dog_test, k=1)['distance']

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 63000   | 12.3772     | 218.199ms    |

| Done         | 509000  | 100         | 265.273ms    |

+--------------+---------+-------------+--------------+

In [63]:
dog_dog_neighbors = dog_model.query(dog_test, k=1)['distance']

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 63000   | 12.3772     | 296.36ms     |

| Done         | 509000  | 100         | 343.364ms    |

+--------------+---------+-------------+--------------+

In [88]:
dog_distances = graphlab.SFrame({'dog-automobile': dog_automobile_neighbors,
                              'dog-bird': dog_bird_neighbors, 
                              'dog-cat': dog_cat_neighbors, 
                              'dog-dog': dog_dog_neighbors})

In [89]:
dog_distances

dog-automobile,dog-bird,dog-cat,dog-dog
41.9579761457,41.7538647304,36.4196077068,33.4773590373
46.0021331807,41.3382958925,38.8353268874,32.8458495684
42.9462290692,38.6157590853,36.9763410854,35.0397073189
41.6866060048,37.0892269954,34.5750072914,33.9010327697
39.2269664935,38.272288694,34.778824791,37.4849250909
40.5845117698,39.1462089236,35.1171578292,34.945165344
45.1067352961,40.523040106,40.6095830913,39.0957278345
41.3221140974,38.1947918393,39.9036867306,37.7696131032
41.8244654995,40.1567131661,38.0674700168,35.1089144603
45.4976929401,45.5597962603,42.7258732951,43.2422832585


In [94]:
def is_dog_correct(row):
    if row['dog-dog'] == min(row.values()):
        return 1
    else:
        return 0

In [95]:
dog_distances.apply(is_dog_correct).sum()

678

In [96]:
bird_test = image_train[image_train['label'] == 'bird'] 

In [97]:
len(bird_test)

478

In [98]:
dog_train = len(image_train[image_train['label'] == 'dog'])

In [99]:
dog_train

509

In [102]:
cat_train = image_train[image_train['label'] == 'cat']

In [103]:
len(cat_train)

509

In [104]:
auto_train = image_train[image_train['label'] == 'automobile']

In [105]:
len(auto_train)

509

In [108]:
get_images_from_ids(cat_model.query(image_test[0:1], k=1)).show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 6.94ms       |

| Done         |         | 100         | 40.761ms     |

+--------------+---------+-------------+--------------+

In [109]:
get_images_from_ids(dog_model.query(image_test[0:1], k=1)).show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 13.881ms     |

| Done         |         | 100         | 62.858ms     |

+--------------+---------+-------------+--------------+

In [111]:
cat_model.query(image_test[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 5.752ms      |

| Done         |         | 100         | 36.741ms     |

+--------------+---------+-------------+--------------+

36.15573070978294

In [112]:
dog_model.query(image_test[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 14.572ms     |

| Done         |         | 100         | 45.298ms     |

+--------------+---------+-------------+--------------+

37.77071136184157

In [115]:
dog_model.query(image_test, k=1)['distance']

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 4000         | 252000  | 12.3772     | 1.02s        |

| Done         | 2036000 | 100         | 1.17s        |

+--------------+---------+-------------+--------------+

dtype: float
Rows: 4000
[37.46426287842384, 42.3025826355744, 29.34723195850588, 43.138619657492214, 33.47735903726335, 32.845849568405555, 35.03970731890587, 50.511235378316556, 33.90103276968192, 37.484925090925636, 37.83014370145973, 40.24165344091786, 34.945165343981266, 39.09572783446352, 32.25996404749732, 35.38520851881949, 30.04429850876171, 35.4741000423819, 37.67910757833515, 46.84669160445326, 49.58601831413529, 32.58452752259978, 42.73161338540937, 48.73938045492005, 37.65028526143984, 36.980135351208396, 52.379528973638166, 49.44651531656444, 35.15904146550642, 37.769613103220316, 38.39637141071839, 41.12594107067672, 35.10891446032838, 36.78188376911598, 45.60351187400346, 55.503603621864514, 42.05598834700012, 44.91342261967625, 49.09222632540127, 35.52227745087944, 37.75506318625139, 34.57170490702803, 45.40174770695962, 39.40650740098469, 32.76690147472848, 43.242283258453455, 49.7942785586727, 43.69828628867995, 43.872975453343486, 35.84833988829975, 39.04216759238213